Realizando imports

In [ ]:
import pymysql
import json

Conectando à base MySQL

In [ ]:
conn = pymysql.connect(
    host="localhost",
    user="ri_user",
    password="ri123",
    database="ri_documentos"
)
cursor = conn.cursor()


Importando arquivos com o map

In [ ]:
try:
   with open('map_docs.json', 'r', encoding='utf-8') as file:
       map_docs = json.load(file)
   with open('map_queries.json', 'r', encoding='utf-8') as file:
       map_queries = json.load(file)
except FileNotFoundError as e:
    print(f"Erro ao abrir o arquivo: {e}")

In [ ]:
map_docs

Criando tabela para os documentos

In [ ]:
cursor.execute("DROP TABLE IF EXISTS documentos;")
cursor.execute("""
CREATE TABLE documentos (
    doc_id VARCHAR(10),
    texto LONGTEXT,
    FULLTEXT(texto)
) ENGINE=MyISAM;
""")

Limpar tabela

In [ ]:
cursor.execute("TRUNCATE TABLE documentos;")

Verificar tabela

In [ ]:

cursor.execute("SHOW TABLES;")

tabelas = cursor.fetchall()

print("Tabelas no banco:")
for tabela in tabelas:
    print(tabela[0])  

In [ ]:
# Select na tabela de documentos
cursor.execute("SELECT * FROM documentos LIMIT 10;")
result = cursor.fetchall()
for row in result:
    print(row)

Inserindo documentos na base

In [ ]:
for doc_id, texto in map_docs.items():
    cursor.execute("INSERT INTO documentos (doc_id, texto) VALUES (%s, %s);", (doc_id, texto))

Declarando função de consulta

In [ ]:
def buscar_mysql(query, limit):
    sql = """
    SELECT doc_id, texto, MATCH(texto) AGAINST(%s IN NATURAL LANGUAGE MODE) AS score
    FROM documentos
    WHERE MATCH(texto) AGAINST(%s IN NATURAL LANGUAGE MODE)
    ORDER BY score DESC
    LIMIT %s;
    """
    cursor.execute(sql, (query, query, limit))
    resultados = cursor.fetchall()

    return [doc_id for doc_id, _, _ in resultados]  # índice 0-based

Realizando consultas

Teste

In [ ]:
consulta = "violence pandemic pandemic impacted violence society including violent crimes"
ranking = buscar_mysql(consulta, 10)

print("Documentos ranqueados pelo MySQL Full-Text Search:")
length = len(ranking)
for i, doc_id in enumerate(ranking):
    print(f"{i+1}/{length} - {doc_id}")